First I imported my libraries along with my raw data

In [79]:
import pandas as pd
import numpy as np
import seaborn
import datacompy

In [80]:
data_demo = pd.read_csv('data_demo.csv',encoding='ISO-8859–1')
data_diet = pd.read_csv('data_diet.csv',encoding='ISO-8859–1')
data_exam = pd.read_csv('data_exam.csv',encoding='ISO-8859–1')
data_labs = pd.read_csv('data_labs.csv',encoding='ISO-8859–1')
data_meds = pd.read_csv('data_meds.csv',encoding='ISO-8859–1')
data_ques = pd.read_csv('data_ques.csv',encoding='ISO-8859–1')
data_demo_copy = pd.read_csv('data_demo.csv',encoding='ISO-8859–1')
data_diet_copy = pd.read_csv('data_diet.csv',encoding='ISO-8859–1')
data_exam_copy = pd.read_csv('data_exam.csv',encoding='ISO-8859–1')
data_labs_copy = pd.read_csv('data_labs.csv',encoding='ISO-8859–1')
data_meds_copy = pd.read_csv('data_meds.csv',encoding='ISO-8859–1')
data_ques_copy = pd.read_csv('data_ques.csv',encoding='ISO-8859–1')



# Here is where I started seperating data out to condense the hundreds of colums down to something more readable for the initial research process

In [81]:
data_demo_need = data_demo[['SEQN','RIAGENDR','RIDAGEYR','DMDHHSIZ','INDFMPIR']]


In [82]:
data_diet_need = data_diet[['SEQN','DR1DRSTZ','DRQSDIET','DRQSDT1','DRQSDT2','DRQSDT3','DRQSDT4','DRQSDT5',
                            'DRQSDT6','DRQSDT7','DRQSDT8','DRQSDT9','DRQSDT10','DRQSDT11','DRQSDT12','DRQSDT91'
                           ,'DR1TKCAL','DR1TPROT','DR1TCARB','DR1TSUGR','DR1TTFAT','DR1TCHOL','DR1TALCO',
                            'DR1.300','DR1.320Z',]]

#for DR1DRSTZ use only 1 or 4 values as they are reliable

In [83]:
data_exam_need = data_exam[['SEQN','PEASCST1','BPXPULS','BPXSY1','BPXDI1','BPXSY2','BPXDI2','BPXDI3','BPXDI4','BPXSY3','BPXSY4','BMDSTATS','BMXWT',
                           'BMXHT','BMXRECUM']]
#combine BMXHT and BMXRECUM

In [85]:
data_meds_need = data_meds[['SEQN','RXDUSE','RXDRSD1']]
#RXDUSE(takes meds) yes = 2, no = 1, RXDRSD1 = disorders

In [86]:
#data_ques
data_ques_need = data_ques[['SEQN','ALQ130', 'BPD035','BPQ020', 'BPQ080','HSD010','DID040','DIQ160',"DBQ700","DEQ034D"
                           ,'DUQ200','DUQ240','DUQ250','DUQ290','DUQ330','HEQ010','HEQ030','HUQ010','MCQ010','MCQ070','MCQ160A'
                           ,'MCQ160M','MCQ160B','MCQ160C','MCQ160D','MCQ160E','MCQ160F','MCQ160G','MCQ160K','MCQ160L'
                           ,'MCQ160O','MCQ220','MCQ203','OSQ060','PAQ650','PAQ605','PAQ620','PAQ665','SLD010H','SMQ020'
                           ,'SXQ272','SXQ270','SXQ267','SXQ265','SXQ260']]

# I compared the sets to the demographics as my control for participents. This data sheet had the ages so I wanted to see how it compared to the rest. If someones age was not recorded but they showed in any other sets, I did not want to use them.

In [87]:
#compare1 = datacompy.Compare(data_demo_need,data_diet_need,join_columns="SEQN")
#print(compare1.report())

In [88]:
#compare2 = datacompy.Compare(data_demo_need,data_exam_need,join_columns="SEQN")
#print(compare2.report())

In [89]:
#compare3 = datacompy.Compare(data_demo_need,data_meds_need,join_columns="SEQN")
#print(compare3.report())

In [90]:
#compare4 = datacompy.Compare(data_demo_need,data_ques_need,join_columns="SEQN")
#print(compare4.report())


# I then merged together the files to start getting everything together

In [91]:
df = pd.merge(data_demo_need,data_diet_need,how = 'inner',on = 'SEQN')
df = pd.merge(data_exam_need,df,how = 'inner',on = 'SEQN')
df = pd.merge(data_meds_need,df,how = 'inner',on = 'SEQN')
df = pd.merge(data_ques_need,df,how = 'inner',on = 'SEQN')

# I removed duplicates of the participent ID to prevent reuse of a participent


In [92]:
df=df.drop_duplicates(subset=['SEQN'])
df = df.loc[:,~df.columns.duplicated()]

# Merge certain columns to condense down information.

In [93]:
#changed values of DRQSDIET so when summing up, the values are easy to pick out
df['DRQSDIET']=df['DRQSDIET'].replace(to_replace = 1, value = 0)
df['DRQSDIET']=df['DRQSDIET'].replace(to_replace = 2, value = 200)
df['DRQSDIET']=df['DRQSDIET'].replace(to_replace = 9, value = 900)
cols = ['DRQSDIET','DRQSDT1','DRQSDT2','DRQSDT3','DRQSDT5','DRQSDT6','DRQSDT7','DRQSDT8','DRQSDT9','DRQSDT10','DRQSDT11','DRQSDT12','DRQSDT91']
df['Diet Type'] = df[cols].sum(1)
df = df.drop(cols, 1)

#Both are height
cols2 = ['BMXHT', 'BMXRECUM']
df['Height'] = df[cols2].sum(1)

C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/4044629264.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(cols, 1)


# Rename columns and replace values with lables

In [94]:
df['Diet Type']=df['Diet Type'].replace(to_replace = 1.0, value = 'Weight Loss or Low Calorie Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace = 2.0, value = 'Low Fat or Low Cholesterol Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace = 3.0, value = 'Low Salt or Low Sodium Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace = 4.0, value = 'Sugar Free or Low Sugar Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace = 5.0, value  = 'Low Fiber Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace=6.0,value = 'High Fiber Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace=7.0,value = 'Diabetic Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace=8.0,value= 'Weight Gain or Muscle Building Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace= 9.0, value = 'Low Carbohydrate Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace =11.0,value = 'Gluten-Free or Celiac Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace =12.0, value = 'Renal or Kidney Diet')
df['Diet Type']=df['Diet Type'].replace(to_replace =91.0, value ="Other Special Diet")
df['Diet Type']=df['Diet Type'].replace(to_replace =200.0, value ="No Special Diet")
df['Diet Type']=df['Diet Type'].replace(to_replace =900.0, value ="Don't Know")
df['Diet Type']=df['Diet Type'].replace(to_replace =0.0, value ="No Response")


df['Gender']=df['RIAGENDR']
df=df.drop('RIAGENDR',1)
df['Gender']=df['Gender'].replace(to_replace = 1, value = 'Male')
df['Gender']=df['Gender'].replace(to_replace = 2, value = 'Female')

df['Participant ID'] = df['SEQN']
df=df.drop('SEQN',1)

df['Age'] = df['RIDAGEYR']
df=df.drop('RIDAGEYR',1)

df['Household Size'] = df['DMDHHSIZ']
df=df.drop('DMDHHSIZ',1)

df['Income to Poverty Ration'] = df['INDFMPIR']
df=df.drop('INDFMPIR',1)

df['Known Disorders'] = df['RXDRSD1']
df=df.drop('RXDRSD1',1)

df['Takes Medication'] = df['RXDUSE']
df=df.drop('RXDUSE',1)
df['Takes Medication'] = df['Takes Medication'].replace(to_replace = 1,value='No')
df['Takes Medication'] = df['Takes Medication'].replace(to_replace = 2,value='Yes')

df['Days Energy Intake in kcal'] = df['DR1TKCAL']
df=df.drop('DR1TKCAL',1)


df['Days Protein Intake in gm'] = df['DR1TPROT']
df=df.drop('DR1TPROT',1)

df['Days Carbohydrate Intake in gm'] = df['DR1TCARB']
df=df.drop('DR1TCARB',1)

df['Days Sugar Intake in gm'] = df['DR1TSUGR']
df=df.drop('DR1TSUGR',1)

df['Days Fat Intake in gm'] = df['DR1TTFAT']
df=df.drop('DR1TTFAT',1)

df['Days Cholesterol Intake in gm'] = df['DR1TCHOL']
df=df.drop('DR1TCHOL',1)

df['Days Alcohol Intake in gm'] = df['DR1TALCO']
df=df.drop('DR1TALCO',1)

df['Compare to Usual Diet Intake'] = df['DR1.300']
df=df.drop('DR1.300',1)
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = 1,value='Much More Than Usual')
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = 2,value='Usual')
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = 3,value='Less Than Usual')
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = 7,value='No Response')
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = 9,value='Did Not Know)
df['Compare to Usual Diet Intake'] = df['Compare to Usual Diet Intake'].replace(to_replace = NaN,value='Missing')
                                                                                
df['Water Intake'] = df['DR1.320Z']
df=df.drop('DR1.320Z',1)                                                                              
                                                                                
df['Days Alcohol Intake in gm'] = df['BPXPULS']
df=df.drop('BPXPULS',1)

df['Days Alcohol Intake in gm'] = df['BPXSY1']
df=df.drop('BPXSY1',1)
                                                                              
'','BPXSY1','BPXDI1','BPXSY2','BPXDI2','BPXDI3','BPXDI4','BPXSY3','BPXSY4','','BMXWT',
                           


C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/354786015.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('RIAGENDR',1)
C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/354786015.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('SEQN',1)
C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/354786015.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('RIDAGEYR',1)
C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/354786015.py:30: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('DMDHHSIZ',1)
C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/354786015.py:33: FutureWarning: In a future version of

In [95]:
df

,ALQ130,BPD035,BPQ020,BPQ080,HSD010,DID040,DIQ160,DBQ700,DEQ034D,DUQ200,...,DR1TALCO,DR1.300,DR1.320Z,Diet Type,Height,Gender,Participant ID,Age,Household Size,Income to Poverty Ration
0,1.0,62.0,1.0,1.0,2.0,62.0,NaN,2.0,NaN,NaN,...,0.0,2.0,960.0,No Special Diet,171.3,Male,73557,69,3,0.84
2,4.0,53.0,1.0,1.0,4.0,23.0,NaN,3.0,5.0,1.0,...,119.0,2.0,360.0,No Special Diet,176.8,Male,73558,54,4,1.78
6,NaN,40.0,1.0,1.0,3.0,57.0,NaN,3.0,NaN,NaN,...,0.0,2.0,1254.0,Diabetic Diet,175.3,Male,73559,72,2,4.51
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,225.0,No Special Diet,137.3,Male,73560,9,4,2.52
12,NaN,55.0,1.0,2.0,5.0,NaN,2.0,2.0,NaN,NaN,...,0.0,2.0,240.0,Other Special Diet,162.4,Female,73561,73,2,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,3.0,NaN,2.0,2.0,3.0,NaN,2.0,4.0,4.0,2.0,...,14.0,1.0,867.0,No Special Diet,171.3,Male,83727,26,5,NaN
19576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,300.0,No Special Diet,169.4,Female,83728,2,4,1.95
19577,NaN,NaN,2.0,2.0,NaN,NaN,2.0,3.0,5.0,NaN,...,0.0,2.0,1014.0,Weight Loss or Low Calorie Diet,162.3,Female,83729,42,1,3.66
19578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,No Response,119.1,Male,83730,7,4,1.05


In [96]:
indexNames = df[ df['DR1DRSTZ'] == 2 ].index
indexNames2 = df[ df['DR1DRSTZ'] == 5].index
df.drop(indexNames , inplace=True)
df.drop(indexNames2 , inplace=True)
df.drop('DR1DRSTZ',1)
indexNames3 = df[ df['PEASCST1'] == 3 ].index
df.drop(indexNames3, inplace=True)
df.drop('PEASCST1',1)
df

C:\Users\nicol\AppData\Local\Temp/ipykernel_13592/2355940809.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop('DR1DRSTZ',1)


,ALQ130,BPD035,BPQ020,BPQ080,HSD010,DID040,DIQ160,DBQ700,DEQ034D,DUQ200,...,DR1TALCO,DR1.300,DR1.320Z,Diet Type,Height,Gender,Participant ID,Age,Household Size,Income to Poverty Ration
0,1.0,62.0,1.0,1.0,2.0,62.0,NaN,2.0,NaN,NaN,...,0.0,2.0,960.0,No Special Diet,171.3,Male,73557,69,3,0.84
2,4.0,53.0,1.0,1.0,4.0,23.0,NaN,3.0,5.0,1.0,...,119.0,2.0,360.0,No Special Diet,176.8,Male,73558,54,4,1.78
6,NaN,40.0,1.0,1.0,3.0,57.0,NaN,3.0,NaN,NaN,...,0.0,2.0,1254.0,Diabetic Diet,175.3,Male,73559,72,2,4.51
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,225.0,No Special Diet,137.3,Male,73560,9,4,2.52
12,NaN,55.0,1.0,2.0,5.0,NaN,2.0,2.0,NaN,NaN,...,0.0,2.0,240.0,Other Special Diet,162.4,Female,73561,73,2,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19568,NaN,60.0,1.0,1.0,2.0,NaN,2.0,2.0,NaN,NaN,...,0.0,2.0,690.0,No Special Diet,176.0,Male,83724,80,2,3.09
19575,3.0,NaN,2.0,2.0,3.0,NaN,2.0,4.0,4.0,2.0,...,14.0,1.0,867.0,No Special Diet,171.3,Male,83727,26,5,NaN
19576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,300.0,No Special Diet,169.4,Female,83728,2,4,1.95
19577,NaN,NaN,2.0,2.0,NaN,NaN,2.0,3.0,5.0,NaN,...,0.0,2.0,1014.0,Weight Loss or Low Calorie Diet,162.3,Female,83729,42,1,3.66


# Create Age Groups for better visualization later

In [ ]:
labels=['0-9','10-19','20-39','40-59','60-79+']
bins=[0,10,20,40,60,150]
df['Age Group']=pd.cut(df['Age'],bins=bins,labels=labels,right=False) 
#df['Age Group']